In [1]:
import pandas as pd
import numpy as np
import time
from google.colab import drive
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
def discretize(df, target_var):
  for i in list(df):
    if i != target_var:
      col = df[i]
      print(i)
      bins = pd.cut(col, 3, labels = ["1", "2", "3"]) # Discretizes to bins
      df[i] = bins

In [26]:
iris_path = "/content/drive/MyDrive/ML/Lab 8/iris.csv"
target_variable = 'variety'
df = pd.read_csv(iris_path)
discretize(df, target_variable)
y = df[target_variable]
X = df.drop([target_variable], axis = 1)

sepal.length
sepal.width
petal.length
petal.width


In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [45]:
from scipy.io.arff import loadarff 
kick_path = "/content/drive/MyDrive/ML/Lab 8/balanced_kick.csv"
df = pd.read_csv(kick_path)
df.head()

,Unnamed: 0,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,0,0,461,1,2005,5,4,0,74,516,...,9917.333333,11112.333333,1,0,8655,75236,31,9520.666667,0,462.0
1,1,0,188,0,2005,4,3,577,47,474,...,9917.333333,11112.333333,0,0,16926,91752,3,9520.666667,0,2279.0
2,2,0,21,1,2002,7,2,373,42,19,...,0.000000,0.000000,0,0,18880,28625,19,1.000000,0,2279.0
3,3,0,206,1,2007,2,3,443,6,232,...,9917.333333,11112.333333,0,0,3453,94544,3,9520.666667,0,462.0
4,4,0,480,1,2004,6,4,147,50,315,...,9917.333333,11112.333333,0,0,16044,21075,14,9520.666667,0,2279.0


In [46]:
df = df.drop(["WheelType"], axis = 1)
df.head()
target_variable = 'IsBadBuy'
discretize(df, target_variable)



y = df[target_variable]
X = df.drop([target_variable], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


Unnamed: 0
PurchDate
Auction
VehYear
VehicleAge
Make
Model
Trim
SubModel
Color
Transmission
WheelTypeID
VehOdo
Nationality
Size
TopThreeAmericanName
MMRAcquisitionAuctionAveragePrice
MMRAcquisitionAuctionCleanPrice
MMRAcquisitionRetailAveragePrice
MMRAcquisitonRetailCleanPrice
MMRCurrentAuctionAveragePrice
MMRCurrentAuctionCleanPrice
MMRCurrentRetailAveragePrice
MMRCurrentRetailCleanPrice
PRIMEUNIT
AUCGUART
BYRNO
VNZIP1
VNST
VehBCost
IsOnlineSale
WarrantyCost


In [47]:
df.describe()

,IsBadBuy
count,14596.000000
mean,0.122979
std,0.328424
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [61]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
model = AdaBoostClassifier()
start = time.perf_counter()
model.fit(X_train, y_train, 100)
end = time.perf_counter()
y_preds = model.predict(X_test)

In [62]:
corr, tot = 0, len(y_test)
for i, j in zip(list(y_test), list(y_preds)):
  if i == j:
    corr += 1
print("Testing Accuracy:", corr / tot)
print("Training Time:", end - start)
print("\nConfusion Matrix:\n", pd.crosstab(pd.Series(list(y_test)), y_preds))

Testing Accuracy: 0.8815068493150685
Training Time: 5.597193089999564

Confusion Matrix:
 col_0     0   1
row_0          
0      2493  75
1       271  81


In [50]:
print(list(X_train))

['Unnamed: 0', 'PurchDate', 'Auction', 'VehYear', 'VehicleAge', 'Make', 'Model', 'Trim', 'SubModel', 'Color', 'Transmission', 'WheelTypeID', 'VehOdo', 'Nationality', 'Size', 'TopThreeAmericanName', 'MMRAcquisitionAuctionAveragePrice', 'MMRAcquisitionAuctionCleanPrice', 'MMRAcquisitionRetailAveragePrice', 'MMRAcquisitonRetailCleanPrice', 'MMRCurrentAuctionAveragePrice', 'MMRCurrentAuctionCleanPrice', 'MMRCurrentRetailAveragePrice', 'MMRCurrentRetailCleanPrice', 'PRIMEUNIT', 'AUCGUART', 'BYRNO', 'VNZIP1', 'VNST', 'VehBCost', 'IsOnlineSale', 'WarrantyCost']


In [51]:
class Node(object):
    def __init__(self, attr):
      self.leaf = False
      self.out = -1
      self.attr_value = attr
      self.children = []
    def set_attribute(self, value):
      self.attr_value = value
    def is_leaf(self, value):
      self.leaf = True
      self.out = value
    def add_child(self, obj):
      self.children.append(obj)


In [52]:
from math import log2, inf
class DecisionTreeOfRandomForests:
  def __init__(self):
      self.root_node = Node("")
      self.max_depth = 0
  def entropy(self, X_c, y_c):
    val = 0
    for i in y_c.value_counts():
      val += (i / len(y_c)) * log2(i / len(y_c))
    return val * -1
  def split_by_attr(self, X, y, attr):
    full_dset = X_train.join(y_train)
    vals = []
    for i in range(1, 4):
      x_attr = full_dset.loc[full_dset[attr] == str(i)].drop([target_variable], axis = 1)
      y_attr = full_dset.loc[full_dset[attr] == str(i)][target_variable]
      vals.append((x_attr, y_attr))
    return vals
  def recur(self, X_s, y_s, remaining_attributes, node, depth = 0):
    if(depth > self.max_depth or len(remaining_attributes) == 0):
      l = y_s.value_counts()
      outp = l[l == l.max()].index.values[0]
      node.is_leaf(outp)
      return
    best_children = []
    best_attribute = ""
    max_gain = -inf
    parent_info = self.entropy(X_s, y_s)
    if(parent_info == 0):
      l = y_s.unique()
      node.is_leaf(l[0])
      return
    for att in remaining_attributes:
      children = self.split_by_attr(X_s, y_s, att)
      children_entropy = 0
      for i in children:
        X_c, y_c = i[0], i[1]
        child_entropy = self.entropy(X_c, y_c)
        children_entropy += (len(X_c) / len(X_s)) * child_entropy
      #print(parent_info, children_entropy, att)
      gain = parent_info - children_entropy
      if gain > max_gain:
        best_attribute = att
        max_gain = gain
        best_children = children
    #print(max_gain, best_attribute)
    remaining_attributes.remove(best_attribute)
    temp = remaining_attributes.copy()
    count = 0
    if max_gain < 0 and depth > 1:
      l = y_s.value_counts()
      outp = l[l == l.max()].index.values[0]
      node.is_leaf(outp)
      return
    for i in best_children:
      #print(depth, count, len(i[0]), temp)
      node.set_attribute(best_attribute)
      n = Node("")
      node.add_child(n)
      rfc = RandomForestClassifier()
      rfc.fit(i[0].values, i[1])
      n.is_leaf(rfc)
  def fit(self, X, y):
    self.max_depth = len(list(X))
    self.recur(X, y, list(X), self.root_node)
    return self.root_node
  def predict(self, X):
    preds = []
    for i in X.iterrows():
      preds.append(self.predict_helper(i[1], i[0]))
    return pd.Series(preds)
  def predict_helper(self, X, idx):
    return self.predict_recur(X, self.root_node, idx)
  def predict_recur(self, X, node, idx):
    if(node.leaf):
      a = node.out.predict(X.array.reshape(1, -1))
      return a[0]
    return self.predict_recur(X, node.children[int(X.loc[node.attr_value]) - 1], idx)


In [53]:
import math
class RandomForestOfRandomForests:
  def __init__(self):
    self.ntrees = 0
    self.trees = []
  def bagging(self, X_p, y_p):
    full_dset = X_p.join(y_p)
    full_dset = full_dset.sample(n=len(X_p) // (2),replace=True)
    X_p = full_dset.drop([target_variable], axis = 1)
    y_p = full_dset[target_variable]
    X_p = X_p.sample(n=int(math.sqrt(len(list(X_p))) + 1),axis='columns')
    return X_p, y_p
  def fit(self, X, y, ntrees = 4):
    self.ntrees = ntrees
    for i in range(ntrees):
      X_p, y_p = self.bagging(X, y)
      tree = DecisionTreeOfRandomForests()
      tree.fit(X_p, y_p)
      self.trees.append(tree)
  def predict(self, X):
    preds = []
    for tree in self.trees:
      pred = tree.predict(X)
      preds.append(pred)
    df_preds = pd.concat(preds, axis=1).T
    df_preds.head()
    return df_preds.mode().iloc[0]

In [54]:
rf_rf = RandomForestOfRandomForests()
start = time.perf_counter()
rf_rf.fit(X_train, y_train, 10)
end = time.perf_counter()

In [55]:
y_preds = rf_rf.predict(X_test)

In [57]:
corr, tot = 0, len(y_test)
for i, j in zip(list(y_test), list(y_preds)):
  if i == j:
    corr += 1
print("Testing Accuracy:", corr / tot)
print("Training Time:", end - start)
print("\nConfusion Matrix:\n", pd.crosstab(pd.Series(list(y_test)), y_preds))

Testing Accuracy: 0.8886986301369864
Training Time: 11.307367295000404

Confusion Matrix:
 0       0.0  1.0
row_0           
0      2481   87
1       238  114


In [24]:
options_per_feature = {'sepallength': {1., 2., 3.},
                       'sepalwidth': {1., 2., 3.},
                       'petallength': {1., 2., 3.},
                       'petalwidth': {1., 2., 3.}}# {feature: set(options) for feature, options in train_dataset.items()}
to_predict = 'class'

import random
class Node(object):
    def __init__(self, name, depth, leaf=False):
        self.children = None if leaf else []
        self.feature_name = name
        self.depth = depth
        self.isLeaf = leaf
        self.next_options = {f_name: None
                             for f_name in options_per_feature[self.feature_name]} \
            if not leaf else None

    def predict(self, x_vals):
        if self.next_options[x_vals[self.feature_name]] is None:
            temp_node = None
            while temp_node is None:
                temp_node = self.next_options[random.sample(self.next_options.keys(), 1)[0]]
            next_node = temp_node
        else:
            next_node = self.next_options[x_vals[self.feature_name]]
        while not next_node.isLeaf:
            if next_node.next_options[x_vals[next_node.feature_name]] is None:
                temp_node = None
                while temp_node is None:
                    temp_node = next_node.next_options[random.sample(next_node.next_options.keys(), 1)[0]]
                # while temp_node is None:
                #     temp_node = next_node.next_options[random.sample(next_node.next_options.keys(), 1)[0]]
                next_node = temp_node
            else:
                next_node = next_node.next_options[x_vals[next_node.feature_name]]
        return next_node.feature_name

    def print(self):
        this_indent = 3 * 2 * self.depth * ' ' + '* '
        options_indent = 3 * (2 * self.depth + 1) * ' ' + '* '
        print(this_indent + self.feature_name)
        for f_name, next_node in self.next_options.items():
            if next_node is not None:
                if next_node.isLeaf:
                    print(options_indent + f_name, '-->', next_node.feature_name)
                    continue
                print(options_indent + f_name)
                next_node.print()

    def make_text_file(self, out_filename='treeout.txt'):
        original_stdout = sys.stdout
        with open(out_filename, 'w') as fout:
            sys.stdout = fout
            self.print()
        sys.stdout = original_stdout

    def add_child(self, option, next_node):
        self.next_options[option] = next_node


def train_model(node, option_name, curr_data):
    if get_entropy(curr_data) == 0.0:
        # unique = set(curr_data[node.feature_name])
        child = Node(curr_data[to_predict][0], node.depth+1, leaf=True)
        node.add_child(option_name, child)
        return
    best, inf_gained = choose_best_feature(curr_data)
    if inf_gained == 0.0:
        # unique = set(curr_data[node.feature_name])
        child = Node(random.choice(curr_data[to_predict]), node.depth+1, leaf=True)
        node.add_child(option_name, child)
        return
    unique = set(curr_data[best])
    child = Node(best, node.depth+1)
    node.add_child(option_name, child)
    for item in unique:
        trimmed_set = trim_dataset(best, item, curr_data)
        train_model(child, item, trimmed_set)


def get_entropy(curr_data):
    last_col = curr_data[to_predict]
    distinct = set(last_col)
    counts = {name: 0 for name in distinct}
    for val in last_col:
        counts[val] += 1
    total = [(freq / len(last_col)) * (log(freq / len(last_col), 2))
             for name, freq in counts.items()]
    return -1*sum(total)


def expected_entropy(feature_name, curr_data):
    this_col = curr_data[feature_name]
    distinct = set(this_col)
    counts = {name: 0 for name in distinct}
    for val in this_col:
        counts[val] += 1
    total = [(counts[val]/len(this_col))*get_entropy(trim_dataset(feature_name, val, curr_data))
             for val in distinct]
    return sum(total)


def trim_dataset(feature_name, value_name, curr_data):
    new_dataset = {f_name: [val for index, val in enumerate(curr_data[f_name])
                            if curr_data[feature_name][index] == value_name]
                   for f_name in curr_data.keys()}
    return new_dataset


input_features = ['sepallength', 'sepalwidth', 'petallength', 'petalwidth']


def choose_best_feature(curr_data):
    orig_entropy = get_entropy(curr_data)
    max_inf_gained = -1
    best_feature = None
    for f_name in input_features:
        expected = expected_entropy(f_name, curr_data)
        inf_gained = orig_entropy-expected
        if inf_gained > max_inf_gained:
            max_inf_gained = inf_gained
            best_feature = f_name
    return best_feature, max_inf_gained


def train_decision_tree(x_train, print_tree=False):
    start, inf_gained = choose_best_feature(x_train)
    root = Node(start, 0)
    unique_items = set(x_train[start])
    for item_name in unique_items:
        small_data = trim_dataset(start, item_name, x_train)
        train_model(root, item_name, small_data)
    if print_tree:
        root.print()
    return root